In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from Multiband import Multiband
from MPbandTool import MPbandTool
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8,14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001   
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g 
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [4]:
mb=Multiband(phase,[80,180],straight_get,p,d,lower,upper,speedVar[0], speedVar[1],vol,px)
mb._solve()

13 13
conflict(s): 0
status  = integer optimal solution
time    = 0.015 s.
problem = MILP
gap     = 0%

object value: 2103.509


In [5]:
a=mb.get_dataframe()
a

,cross_number,distance,sg1,sg2,b1,b2,offset,p,t1,t2,z,u1,u2,w1,car_t1,w2,car_t2
0,1.0,800.0,0.282,0.330,0.282,0.330,0.000000,0.0,83.677143,73.282857,80.000000,0.000000,0.000000,0.1410,11.139000,0.1650,35.595000
1,2.0,520.0,0.426,0.368,0.260,0.330,72.157143,0.0,37.142857,50.777143,80.000000,0.000000,0.000000,0.2850,94.827143,0.2030,122.312143
2,3.0,500.0,0.260,0.230,0.260,0.230,41.700000,0.0,38.980000,62.500000,80.000000,0.000000,0.000000,0.1300,51.970000,0.1150,71.585000
3,4.0,490.0,0.277,0.277,0.277,0.230,0.000000,0.0,61.250000,61.250000,80.000000,0.000000,0.000000,0.1385,10.941500,0.1150,9.085000
4,5.0,370.0,0.255,0.255,0.255,0.255,0.000000,1.0,36.594881,46.250000,136.933687,0.223375,0.221875,0.1275,17.331545,0.1275,17.331545
5,6.0,254.0,0.650,0.650,0.243,0.255,36.594881,0.0,22.338806,31.750000,136.933687,0.000000,0.000000,0.1275,53.932426,0.5225,108.015232
6,7.0,585.0,0.243,0.243,0.165,0.243,59.755289,0.0,71.837984,65.095703,136.933687,0.000000,0.000000,0.1215,76.310232,0.1215,76.271232
7,8.0,1020.0,0.165,0.165,0.165,0.165,0.000000,0.0,127.500000,72.857143,136.933687,0.000000,0.000000,0.0825,11.214529,0.0825,11.214529
8,9.0,409.0,0.298,0.298,0.298,0.298,32.115000,1.0,51.125000,32.115000,80.000000,0.536830,0.000001,0.1490,43.886000,0.1490,43.886000
9,10.0,547.0,0.379,0.379,0.379,0.298,0.000000,0.0,68.375000,68.375000,80.000000,0.000000,0.000000,0.1895,14.970500,0.1490,11.771000


In [6]:
a[["b"+str(i+1) for i in range(2)]].mean()

b1    0.274077
b2    0.292538
dtype: float64

In [7]:
mpb=MPbandTool(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p,a.offset/a.z,
            1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
mpb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 0.0000.
Tried aggregator 3 times.
MIP Presolve eliminated 791 rows and 286 columns.
MIP Presolve modified 946 coefficients.
Aggregator did 62 substitutions.
Reduced MIP has 450 rows, 220 columns, and 1136 nonzeros.
Reduced MIP has 35 binaries, 29 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.98 ticks)
Probing fixed 0 vars, tightened 52 bounds.
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 64 rows and 58 columns.
MIP Presolve modified 206 coefficients.
Reduced MIP has 386 rows, 162 columns, and 972 nonzeros.
Reduced MIP has 17 binaries, 29 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.64 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried ag

In [8]:
# colors=[
#             {"color":"gray" , "hatch": None, "fill": True},
#             {"color":"gray", "hatch":"||||" , "fill": True},
#             {"color": "gray", "hatch":"////" , "fill": True},
#             {"color":"white", "hatch": "****", "fill": False},
#             {"color": "white", "hatch": None, "fill": False},
#             {"color": "white", "hatch": "////", "fill": False},
#             {"color": "white", "hatch": "||||", "fill": False},
#             {"color": "white", "hatch": "\\\\\\\\", "fill": False},
#             {"color": "white", "hatch": "xxxx", "fill": False},
#             {"color": "white", "hatch": "....", "fill": False},
# ]
# linestyles=[
#     {"linestyle":"-" },
#     {"linestyle":(5,(10,3))},
#     {"linestyle":(0, (5, 3)) },
#     {"linestyle":":" },
#     {"linestyle":(0, (3, 1, 1, 1)) },
#     {"linestyle": (0, (5, 10)) },
# ]
# legends=["1->15", "1->6", "8->15", "15->2", "14->9", "7->1"]
# mb.draw_car_bound("./时距图",colors,legends,2,linestyles)

In [9]:
a=mpb.get_dataframe()
a

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u3,u4,u5,u6,bb1,bb2,dw1,dw2,tb1,tb2
0,1.0,800.0,0.282,0.282,0.000,0.325421,0.000,0.330,0.000000,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.282,0.330,30.0,30.0,130.000000,130.000000
1,2.0,520.0,0.260,0.260,0.000,0.330000,0.000,0.330,72.157143,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.260,0.330,15.0,15.0,37.142857,37.142857
2,3.0,500.0,0.260,0.260,0.000,0.230000,0.000,0.230,41.700000,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.260,0.230,30.0,30.0,92.500000,92.500000
3,4.0,490.0,0.277,0.277,0.000,0.230000,0.000,0.230,0.000000,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.277,0.230,15.0,15.0,35.000000,35.000000
4,5.0,370.0,0.255,0.000,0.255,0.255000,0.000,0.255,0.000000,1.0,...,0.361875,2.218750e-01,1.000000e-08,2.218750e-01,0.255,0.255,30.0,30.0,76.250000,76.250000
5,6.0,254.0,0.243,0.000,0.243,0.255000,0.000,0.255,36.594881,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.243,0.255,15.0,15.0,18.142857,18.142857
6,7.0,585.0,0.165,0.000,0.165,0.182333,0.000,0.243,59.755289,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.243,30.0,30.0,103.125000,103.125000
7,8.0,1020.0,0.165,0.000,0.165,0.165000,0.000,0.165,0.000000,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.165,30.0,30.0,157.500000,157.500000
8,9.0,409.0,0.298,0.000,0.298,0.298000,0.298,0.000,32.115000,1.0,...,0.536830,1.000000e-08,8.250100e-02,1.000000e-08,0.298,0.298,30.0,30.0,81.125000,81.125000
9,10.0,547.0,0.379,0.000,0.379,0.298000,0.298,0.000,0.000000,0.0,...,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.379,0.298,30.0,30.0,98.375000,98.375000


In [10]:
a.b1.mean(),a.b2[:8].mean(),a.b3[4:].mean(),a.b4.mean(),a.b5[7:].mean(),a.b6[:8].mean(),a.bb1.mean(),a.bb2.mean()

(0.27407692307692305,
 0.13487500000000002,
 0.2760000000115898,
 0.2875195984468187,
 0.29416666666666663,
 0.2547500000025611,
 0.27407692307692305,
 0.2925384615384615)

In [11]:
a.y3

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
Name: y3, dtype: float64